# EO over Ghana fields

In this notebook, we will show the Earth observation (EO) data over ground measurement fields and show how we interpret the satellite signals into variables (biophysical parameters) that closely describes the growth conditions of maize in the field. 

# Background

## Biophysical parameters

Biophysical, biochemical and phenological traits of plants are linked to the photosynthesis and nutrition processes during the growth cycle. They control response to the abiotic and biotic environmental filters, and in turn they feed back on the ecosystem and climate system through exchange of matter and energy cycle in the interaction between biosphere and atmosphere. Understanding traits and their co-variations lays a mechanistic foundation for quantitative and predictive analysis of planet ecology and change and can also help guide and refine functional ecosystem clustering. Modelling growth from a viewpoint of traits and their environmental interactions can provide a robust basis to link plants and ecosystem dynamics and ecosystem services such as the water, energy and other fluxes and help us to determine how climate and vegetation interact over a large range of timescales.

## Optical satellite measurements

Satellites like Sentibel 2 (S2) and Landsat-8 (L8) scan the Earth surface with optical sensors and the measurements are expressed as radiance or reflectance. But it is hard to link them directly to the crops growing status in the field. Traditionally, regression analysis can be used to build empirical relationships between variables expressing the crop growth conditions to the reflectance/radiance or ratios like NDVI. But those empirical relationships can be hardly extrapolated spatially or temporally. 

## Radiative transfer models

To systematically and physically interpret those satellite measurements, more advanced radiative transfer theory, physically expressing the satellite signal as a function of biophysical parameters, is normally used. With reasonable simplification and approximation of the canopy of crops, radiative transfer models are able to forward model the satellite measurements with different combinations of biophysical parameters. In this project, one of the mostly studied 1D canopy radiative transfer model is used to helps us to interpret the satellite signals. It consists of a leaf model PROSPECT and a canopy bidirectional reflectance model, SAIL (Scattering by Arbitrary Inclined Leaves), and the coupling of those two helps us the simulate satellite signals as a set of leaf parameters and canopy structure parameters. It mainly includes following parameters:

| Traits type        | Parameter                        | Symbol      | Range      | Unit         |
|--------------------|----------------------------------|-------------|------------|--------------|
| Canopy structure   | leaf area index                  | $LAI$       | 0-8        | m²/m²        |
|                    | leaf angle distribution function | $ALA$       | 0-90       | $^{\circ} $  |
|--------------------|----------------------------------|-------------|------------|--------------|
| Leaf optical       | chlorophyll a and b content      | $C_{ab}$    | 0-120      | $\mu g/cm^2$ |
|                    | Carotenoid content               | $C_{ar}$    | 0-25       | $\mu g/cm^2$ |
|                    | Anthocyanin content              | $C_{an}$    | 0          | $\mu g/cm^2$ |
|                    | leaf dry matter per leaf area    | $C_{m}$     | 0.002-0.02 | $\mu g/cm^2$ |
|                    | leaf water content per leaf area | $C_{w}$     | 0-0.04     | mg/cm²       |
|                    | brown pigment content            | $C_{brown}$ | 0-1        | -            |
|                    | mesophyll structure coefficient  | $N$         | 1-2.5      | -            |
|--------------------|----------------------------------|-------------|------------|--------------|
| angles             | solar zenith angle               | sza         | 0-80       | $^{\circ} $  |
|                    | viewing zenith angle             | vza         | 0-15       | $^{\circ} $  |
|                    | relative azimuth angle           | raa         | 0-360      | $^{\circ} $  |


## Time series satellite signal interpretation

Normally, time series of satellite signals are used to analysis the growth behaviour rather than individual date's observations. We can easily apply our radiative transfer model to any date's observation and try to invert those parameters using different methods, such as minimizing a cost function expressing the difference between the model simulated spectra and the satellite measurements, or machine learning methods, create a response function from the modelled reflectance to the individual parameters. Many studies have shown a certain level of accuracy can be achieved, but normally there are issues in inverting those parameters having smaller effects on the modelled spectra. This is normally attributed to either lack of enough observation information to constrain the inverse problem or the confounding effects in the radiative transfer model, i.e. different combination of bio-physical parameters generating spectra that are almost identical. Normally, prior information are used to better constrain the problem, which can be derived from different products, preivous studies and so on. 

In this demonstration, we uses prior information derived from China and US and they describe the typical temporal evolution of biophysical parameters throughout the crop growing season. We call them the archetypes of biophysical parameters. It contains the temporal changes of biophysical parameters, which are superiour than the normal constant values or range of values with uniform or Gaussian distributions. With additional four phenology parameters and 7 scaling parameters, we can simulate all kinds of growing behaviour of crops, to mimic crop growth under different conditions.



Feng - calibration of lai etc from the EO. That’s done really well for a first pass, but think about how it could be refined. How about fitting to VIs as well as reflectance? Does that help at all to narrow the posteriors? What about a 2-pass sampling we discussed? What about looking more at individual pixels in the field. How much variation is there?
Feng - Obv an empirical model between your parameters and yield would be interesting- see if the relationship could be predicted from your US calibration or vice-versa? But maybe they measure yield differently. See how well-constrained a max-lai and phenology prediction for yield is. 

In [1]:
from osgeo import ogr
fields = '../data/carto/Biophysical_Data_Collection_Polygons_V1.geojson'

g = ogr.Open(fields)
l = g.GetLayer(0)
l.GetFeatureCount

<bound method Layer.GetFeatureCount of <osgeo.ogr.Layer; proxy of <Swig Object of type 'OGRLayerShadow *' at 0x7f35045b62d0> >>

In [3]:
import folium
import requests
from osgeo import ogr
from pygeotile.tile import Tile

zoom = 17

fields = '../data/carto/Biophysical_Data_Collection_Polygons_V1.geojson'

g = ogr.Open(fields)



my_map = folium.Map(location=[9.386, -0.688], zoom_start=zoom)
tile = Tile.for_latitude_longitude(*my_map.location, zoom)
x, y = tile.tms

folium.TileLayer('openstreetmap').add_to(my_map)

for i in range(x-2, x+4):
    for j in range(y - 3, y + 4):
        tile = Tile.from_tms(i, j, zoom)
        url = "http://ecn.t3.tiles.virtualearth.net/tiles/a%s.png?g=1"%tile.quad_tree
        ul, br = tile.bounds
        folium.raster_layers.ImageOverlay(
            image=url,
            name='sample map',
            opacity=1,
            bounds=[[br.latitude, ul.longitude], [ul.latitude, br.longitude]],
            interactive=False,
            cross_origin=False,
            zindex=1,
        ).add_to(my_map)
        
folium.GeoJson('../data/Biophysical_Data_Collection_Polygons_V1.geojson').add_to(my_map)
my_map

In [3]:
from ipyleaflet import *
from ipywidgets import *
import numpy as np
import requests
from pygeotile.tile import Tile

zoom = 16

my_map = Map(center=(9.3771, -0.6062), zoom=zoom, scroll_wheel_zoom=True)
tile = Tile.for_latitude_longitude(*my_map.center, zoom)
x, y = tile.tms

for i in range(x-3, x + 4):
    for j in range(y - 3, y + 4):
        tile = Tile.from_tms(i, j, zoom)
        url = "http://ecn.t3.tiles.virtualearth.net/tiles/a%s.png?g=1"%tile.quad_tree
        ul, br = tile.bounds
        image = ImageOverlay(
            url=url,
            bounds = tile.bounds,
            name = 'bing_basemap_%d'%zoom
        )
        my_map.add_layer(image)  
        
        
def on_change_zoom(change):
    if change['type'] == 'change' and change['name'] == 'zoom':
        zoom = int(my_map.zoom)
        tile = Tile.for_latitude_longitude(*my_map.center, zoom)
        x, y = tile.tms
        

        for i in range(x- 3, x + 4):
            for j in range(y - 2, y + 3):
                tile = Tile.from_tms(i, j, zoom)
                url = "http://ecn.t3.tiles.virtualearth.net/tiles/a%s.png?g=1"%tile.quad_tree
                image = ImageOverlay(
                    url=url,
                    bounds = tile.bounds,
                    name = 'bing_basemap_%d'%zoom
                )
                my_map.add_layer(image)   

        for layer in my_map.layers:
            if layer.name == 'bing_basemap_%d'%int(change['old']):
                my_map.remove_layer(layer)
        
my_map.observe(on_change_zoom)


with open('data/Biophysical_Data_Collection_Polygons_V1.geojson', 'r') as f:
    data = json.load(f)

field_ids = [feat['properties']['Field_ID'] for feat in data['features']]


dropdown = Dropdown(
    options=field_ids,
    value=field_ids[0],
    description="Field ID:",
)



def on_click(change):
    global data_name

    data_name = change["new"]
    
    ind = field_ids.index(data_name)
    
    feature = data['features'][ind]
    poly = geometry.Polygon(feature['geometry']['coordinates'][0])
    lon, lat = poly.centroid.coords[0]
    my_map.center = lat, lon
    zoom = 17
    tile = Tile.for_latitude_longitude(*my_map.center, zoom)
    x, y = tile.tms


    for i in range(x- 3, x + 4):
        for j in range(y - 2, y + 3):
            tile = Tile.from_tms(i, j, zoom)
            url = "http://ecn.t3.tiles.virtualearth.net/tiles/a%s.png?g=1"%tile.quad_tree
            image = ImageOverlay(
                url=url,
                bounds = tile.bounds,
                name = 'bing_basemap_%d'%zoom
            )
            my_map.add_layer(image)   

    
    
    
dropdown.observe(on_click, "value")
widget_control2 = WidgetControl(widget=dropdown, position="bottomleft")



my_map.add_control(widget_control2)



def random_color(feature):
    return {
        'color': 'black',
        'fillColor': np.random.choice(['red', 'yellow', 'green', 'orange']),
    }

geo_json = GeoJSON(
    data=data,
    style={
        'opacity': 1, 'dashArray': '0', 'fillOpacity': 0.1, 'weight': 1
    },
    hover_style={
        'color': 'white', 'dashArray': '0', 'fillOpacity': 0.5
    },
    style_callback=random_color
)


label = Label()
display(label)

def handle_interaction(**kwargs):
    if kwargs.get('type') == 'mousemove':
        label.value = str(kwargs.get('coordinates'))

my_map.on_interaction(handle_interaction)


my_map.add_layer(geo_json)
my_map

ModuleNotFoundError: No module named 'pygeotile'